# Estudo inicial sobre qualidade da água no Lago Guaíba a partir de imagens de satélite

O presente notebook documenta a exploração inicial realizada para o estudo sobre qualidade da água no Lago Guaíba a partir de imagens de satélite, projeto realizado durante a mentoria do [Alforriah](https://www.alforriah.com/) durante o primeiro semestre de 2022.

In [1]:
import ee
import pandas as pd
import geopandas as gpd
import json
import geemap
import geemap.colormaps as cm


Autenticar GEE

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWhpxJR2Act5ksF7ILPxC1H3bUI04eQvJ2nUskjmTXHcayz7lDjQ6uk

Successfully saved authorization token.


## 1. Exploração Google Earth Engine

Esse projeto foi minha experiência utilizando o Google Earth Engine, assim foi realizado inicialmente a análise de algumas funções da API. Grande parte do código foi auxiliado pelo curso fornecido por Cristian Cunha e pelo material disponibilizado no seu canal de YouTube.

### 1.1. Extraindo uma imagem do Guaíba

Inicalmente é realizada a leitura da área de estudo, de modo que seja possível extrair as imagens de satélite para a região de interesse, a partir da criação de uma `Feature Collection`.

In [4]:
# Leitura do arquivo shapefile da regiao
gdf = gpd.read_file("../../data/external/bcrs25/guaiba.shp") # geodataframe
gdf.loc[:, "nome"] = "Lago Guaíba" #corrigir o nome

# converter de shp para json
roi = gdf.to_json()

#carrega arquivo json
roi = json.loads(roi)

#o arquivo roi consiste em um dicionário. estamos interessados na key "features"
roi = roi["features"]

#criamos uma FeatureCollection (https://developers.google.com/earth-engine/guides/feature_collections)
region = ee.FeatureCollection(roi)

Extrai-se uma `ImageCollection` (pilha de imagens sequenciais) da área de estudo. Trabalhou-se com o satélite Sentinel 2-A (https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR), sendo disponibilizados dados desde 28/03/2017. As diferentes imagens serâo ordenadas pelo percentual de pixels associados a nuvens, selecionando-se a imagem mais limpa da série, recortando para a área de interesse.

In [5]:
guaiba_img = ee.ImageCollection("COPERNICUS/S2_SR")\
    .filterBounds(region)\
    .sort("CLOUDY_PIXEL_PERCENTAGE")\
    .first()\
    .clip(region)

In [5]:
guaiba_img.get("CLOUDY_PIXEL_PERCENTAGE").getInfo()

0.002317

In [6]:
import datetime

data_da_img = datetime.datetime\
    .fromtimestamp(guaiba_img.date().millis().getInfo()/1000)\
    .strftime("%Y-%m-%d")

print(f"imagem do dia {data_da_img}")

imagem do dia 2022-02-16


In [7]:
Map_01 = geemap.Map()

Map_01 = geemap.Map(location=[-30.208419, -51.255901], zoom_start=10)
Map_01.add_basemap("HYBRID")

Map_01.addLayer(guaiba_img, {"bands": ["B4", "B3", "B2"],
             "min": 0,
             "max": 5000},
             "Sentinel 2")

display(Map_01)

Map(center=[-30.208419, -51.255901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

### 1.2. Calculando NDCI para uma única imagem

Para os cálculos dos índices, faz-se necessário multiplicar a reflectância por 0.0001, considerando que as bandas foram escaladas.

https://gis.stackexchange.com/questions/324489/sentinel-2-surface-reflectance-range

In [8]:
img_reflectancia = guaiba_img.multiply(0.0001) 

O Normalized Difference Chlorophyll Index (NDCI) consiste em um índice utilizado para a identificação de locais com floração de algas, identificando-se alguns trabalhos que já foram feitos a partir dele, por exemplo:

https://www.mdpi.com/2072-4292/13/15/2874/htm

http://www.pjoes.com/pdf-98994-42186?filename=Assessing%20Spectral.pdf

O índice é calculado a partir da seguinte expressão:

$$\mathrm{NDCI} = \frac{\rho_{\mathrm{V1}} - \rho_{\mathrm{R}}}{\rho_{\mathrm{V1}} + \rho_{\mathrm{R}}}$$

Em que $\rho_{\mathrm{R}}$ consiste na banda do vermelho visível (B4) e $\rho_{\mathrm{V1}}$ na "red edge" (B5)

O trabalho também irá considerar o valor de NDVI, em que $\rho_{\mathrm{NIR}}$ consiste no infravermelho próximo

$$\mathrm{NDVI} = \frac{\rho_{\mathrm{NIR}} - \rho_{\mathrm{R}}}{\rho_{\mathrm{NIR}} + \rho_{\mathrm{R}}}$$

In [9]:
ndci_img = img_reflectancia.normalizedDifference(["B5", "B4"]).rename("NDCI")
ndci_viz = {"min":-1,"max":1,"palette":["black","green", "yellow","red"]}

Map_01.addLayer(ndci_img, ndci_viz, "NDCI")
display(Map_01)

Map(center=[-30.208419, -51.255901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

Verifica-se que os maiores valores de NDCI para a imagem ocorrem junto à porção de terra situada mais ao norte do Guaíba. Além disso, como esperado, a distribuição dos valores de NDCI não é uniforme ao longo do Lago Guaíba.

Considerando a presença de trecho de terra, busca-se sua remoção a partir da aplicação de máscara a partir dos índices NDWI (Normalized Difference Water Index) e NDVI (Normalized Difference Vegetation Index).

In [10]:
# Mask de água
ndwi = img_reflectancia.normalizedDifference(["B3", "B8"]).rename("NDWI")
ndvi = img_reflectancia.normalizedDifference(["B8", "B4"]).rename("NDVI")
mask_agua = ndvi.lte(0).And(ndwi.gte(0))

In [11]:
ndci_masked = ndci_img.updateMask(mask_agua)
Map_01.addLayer(ndci_masked, ndci_viz, "NDCI masked")
display(Map_01)

Map(center=[-30.208419, -51.255901], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…

## 2. Trabalhando com séries temporais

Busca-se agora extrair uma coleção de imagens para a área de estudo.

In [12]:
guaiba_collection = ee.ImageCollection("COPERNICUS/S2_SR")\
    .filterBounds(region)\
    .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 20))

n_images = guaiba_collection.size().getInfo()
date_range = guaiba_collection.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])

init_date = datetime.datetime\
    .fromtimestamp(date_range.get("min")\
    .getInfo() / 1000)\
    .strftime("%Y-%m-%d")

last_date = datetime.datetime\
    .fromtimestamp(date_range.get("max")\
    .getInfo() / 1000)\
    .strftime("%Y-%m-%d")

print(f"{n_images} imagens entre {init_date} e {last_date}")

171 imagens entre 2018-12-29 e 2022-05-12


In [23]:
def add_id_date(img):
    """Adiciona ID e data da imagem Sentinel 2A"""

    return img.set({"ID": img.get("system:id")})\
                .set({"millis": img.date().millis()})\
                .set("date", img.date().format())

def mask_s2a_clouds(img):
    """Adiciona máscara de núvens em imagem Sentinel 2A, retornando bandas B*

    Os bits 10 e 11 são nuvens e cirros, respectivamente.
    https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR
    Créditos: Scripts Remote (Me. Christhian Cunha - https://linktr.ee/scriptsremotesensing)"""

    qa60 = img.select('QA60')
    
    cloudBitMask = 1 << 10                              # Bit 10 corresponde a nuvens
    cirrusBitMask = 1 << 11                             # Bit 11 corresponde a cirrus
    
    mask = qa60.bitwiseAnd(cloudBitMask).eq(0)and(qa60.bitwiseAnd(cirrusBitMask).eq(0))

    return img.updateMask(mask)\
      .select("B.*")\
      .copyProperties(img, img.propertyNames())

def add_s2a_ndci_ndvi(img):
    """Adiciona NDCI e NDVI para imageCollection Sentinel 2A
    
    O cálculo dos índices considera as seguintes referências:
    - Jaskula e Sojka (2019): http://www.pjoes.com/pdf-98994-42186?filename=Assessing%20Spectral.pdf
    - Doc. Sentinel 2A GEEhttps://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#bands
    - Lobo et al (2021): https://www.mdpi.com/2072-4292/13/15/2874/html"""
    
    img_reflectancia = img.multiply(0.0001)
    ndci_band = img_reflectancia.normalizedDifference(["B5", "B4"]).rename("NDCI")
    ndvi_band = img_reflectancia.normalizedDifference(["B8", "B4"]).rename("NDVI")
    
    img_with_bands = img.addBands([ndci_band, ndvi_band])\
        .copyProperties(img, ["system:time_start"])\
        .set("date", img.date().format("YYYY-MM-dd"))
    
    return img_with_bands

In [17]:
def clp_region(img):
    return img.clip(region)

In [18]:
guaiba_collection_ndvi_ndci = guaiba_collection.map(mask_s2a_clouds).map(clp_region).map(add_s2a_ndci_ndvi).map(add_id_date)

In [ ]:
#verificar se NDVI e NDCI foram adicionados

guaiba_collection_ndvi_ndci.first().bandNames().getInfo()

### 2.1. Criando GIF

In [20]:
import os

In [20]:
gif_dir = "../../data/processed/gee-gif/ndci"

if os.listdir(gif_dir) == []:
    geemap.ee_export_image_collection(guaiba_collection_ndvi_ndci.select("NDCI"),
                                  out_dir=gif_dir,
                                  crs="EPSG:4674",
                                  region=region.geometry())

In [21]:
gif_dir = "../../data/processed/gee-gif/ndvi"

if os.listdir(gif_dir) == []:
    geemap.ee_export_image_collection(guaiba_collection_ndvi_ndci.select("NDVI"),
                                  out_dir=gif_dir,
                                  crs="EPSG:4674",
                                  region=region.geometry())

Total number of images: 171

Exporting 1/171: 20181229T133221_20181229T133218_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20181229T133221_20181229T133218_T22JDM.tif


Exporting 2/171: 20190125T132231_20190125T132230_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20190125T132231_20190125T132230_T22JDM.tif


Exporting 3/171: 20190128T133221_20190128T133221_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20190128T133221_20190128T133221_T22JDM.tif


Exporting 4/171: 20190207T133221_20190207T133221_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20190207T133221_20190207T133221_T22JDM.tif



Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20190712T133239_20190712T134011_T22JDM.tif


Exporting 21/171: 20190803T132241_20190803T132910_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20190803T132241_20190803T132910_T22JDM.tif


Exporting 22/171: 20190806T133231_20190806T133228_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20190806T133231_20190806T133228_T22JDM.tif


Exporting 23/171: 20190811T133229_20190811T133958_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20190811T133229_20190811T133958_T22JDM.tif


Exporting 24/171: 20190902T132231_20190902T132233_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Us

Exporting 40/171: 20191219T133219_20191219T133219_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20191219T133219_20191219T133219_T22JDM.tif


Exporting 41/171: 20191224T133221_20191224T133219_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20191224T133221_20191224T133219_T22JDM.tif


Exporting 42/171: 20191226T132229_20191226T132409_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20191226T132229_20191226T132409_T22JDM.tif


Exporting 43/171: 20200105T132229_20200105T132227_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20200105T132229_20200105T132227_T22JDM.tif


Exporting 44/171: 202001

Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20200325T132229_20200325T132335_T22JDM.tif


Exporting 60/171: 20200330T132231_20200330T132229_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20200330T132231_20200330T132229_T22JDM.tif


Exporting 61/171: 20200404T132229_20200404T132230_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20200404T132229_20200404T132230_T22JDM.tif


Exporting 62/171: 20200404T132229_20200404T133037_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20200404T132229_20200404T133037_T22JDM.tif


Exporting 63/171: 20200407T133219_20200407T133948_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Us

Exporting 79/171: 20200703T132239_20200703T132235_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20200703T132239_20200703T132235_T22JDM.tif


Exporting 80/171: 20200713T132239_20200713T132234_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20200713T132239_20200713T132234_T22JDM.tif


Exporting 81/171: 20200726T133229_20200726T133226_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20200726T133229_20200726T133226_T22JDM.tif


Exporting 82/171: 20200731T133231_20200731T133230_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20200731T133231_20200731T133230_T22JDM.tif


Exporting 83/171: 202008

Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20210102T133229_20210102T133223_T22JDM.tif


Exporting 99/171: 20210109T132229_20210109T132525_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20210109T132229_20210109T132525_T22JDM.tif


Exporting 100/171: 20210119T132229_20210119T132525_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20210119T132229_20210119T132525_T22JDM.tif


Exporting 101/171: 20210208T132229_20210208T132522_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20210208T132229_20210208T132522_T22JDM.tif


Exporting 102/171: 20210213T132231_20210213T133017_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:

Exporting 118/171: 20210509T132229_20210509T132524_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20210509T132229_20210509T132524_T22JDM.tif


Exporting 119/171: 20210512T133229_20210512T133223_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20210512T133229_20210512T133223_T22JDM.tif


Exporting 120/171: 20210524T132231_20210524T132519_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20210524T132231_20210524T132519_T22JDM.tif


Exporting 121/171: 20210527T133231_20210527T133420_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20210527T133231_20210527T133420_T22JDM.tif


Exporting 122/171: 2

Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20210916T132029_20210916T132817_T22JDM.tif


Exporting 138/171: 20211019T133229_20211019T133226_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20211019T133229_20211019T133226_T22JDM.tif


Exporting 139/171: 20211021T132241_20211021T133005_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20211021T132241_20211021T133005_T22JDM.tif


Exporting 140/171: 20211024T133231_20211024T133229_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20211024T133231_20211024T133229_T22JDM.tif


Exporting 141/171: 20211026T132239_20211026T132614_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C

Exporting 157/171: 20220203T132229_20220203T132227_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20220203T132229_20220203T132227_T22JDM.tif


Exporting 158/171: 20220211T133221_20220211T133524_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20220211T133221_20220211T133524_T22JDM.tif


Exporting 159/171: 20220216T133219_20220216T133218_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20220216T133219_20220216T133218_T22JDM.tif


Exporting 160/171: 20220226T133219_20220226T133220_T22JDM.tif
Generating URL ...
Please wait ...
Data downloaded to C:\Users\Daniel\Documents\Projetos\Alforriah\gee_guaiba\data\processed\gee-gif\ndvi\20220226T133219_20220226T133220_T22JDM.tif


Exporting 161/171: 2

As imagens e o GIF foram gerados em R.

### 2.2. Extrair dados (redução)

In [25]:
def reduce(img):
    """Extrai média, mediana, mínimo, máximo e desvio padrão de uma banda"""
    
    serie_reduce = img.reduceRegions(**{
        "collection":region,
        "reducer": ee.Reducer.mean().combine(**{
            "reducer2": ee.Reducer.min(),
                "sharedInputs": True}).combine(**{
            "reducer2": ee.Reducer.max(),
                "sharedInputs": True}).combine(**{
            "reducer2": ee.Reducer.median(),
                "sharedInputs": True}).combine(**{
            "reducer2": ee.Reducer.stdDev(),
                "sharedInputs":True}),
        "scale": 20
    })
    
    serie_reduce = serie_reduce.map(lambda f: f.set({"millis": img.get("millis")}))\
        .map(lambda f: f.set({"date": img.get("date")}))
    
    return serie_reduce.copyProperties(img, ["system:time_start"])

def create_df(img, band):
    """Cria um dataframe para dados extraidos de imageCollection"""
    
    reduced_img = img.select(band).map(reduce)\
        .flatten()\
        .sort("date", True)\
        .select(["millis", "date", "min", "max","mean", "median", "stdDev"])

    lista_df = reduced_img.reduceColumns(
        ee.Reducer.toList(7),
        ["millis", "date", "min", "max","mean", "median", "stdDev"])\
        .values().get(0)

    df = pd.DataFrame(
        lista_df.getInfo(),
        columns=["millis",
                 "date"] +
        [band + "_" + stat for stat in ["min", "max", "mean", "median", "stdDev"]])
    
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
    
    return df

In [27]:
df_ndvi = create_df(guaiba_collection_ndvi_ndci, "NDVI")

In [29]:
df_ndci = create_df(guaiba_collection_ndvi_ndci, "NDCI")

In [30]:
df_ndci

,millis,date,NDCI_min,NDCI_max,NDCI_mean,NDCI_median,NDCI_stdDev
0,1546090840000,2018-12-29 13:40:40,-0.167931,0.634409,-0.023233,-0.029342,0.034691
1,1548423047000,2019-01-25 13:30:47,-0.180560,0.633309,-0.005937,-0.005875,0.035793
2,1548682843000,2019-01-28 13:40:43,-0.158893,0.647395,-0.010855,-0.013683,0.033420
3,1549546843000,2019-02-07 13:40:43,-0.193239,0.638596,-0.030255,-0.029306,0.040513
4,1549719051000,2019-02-09 13:30:51,-0.176221,0.622495,-0.012386,-0.009748,0.045008
...,...,...,...,...,...,...,...
166,1650807045250,2022-04-24 13:30:45,-0.114764,0.291866,-0.018705,-0.020502,0.020489
167,1651239056300,2022-04-29 13:30:56,-0.111469,0.297750,-0.013373,-0.012695,0.021745
168,1651930843010,2022-05-07 13:40:43,-0.094701,0.294320,-0.009096,-0.008776,0.017226
169,1652103055120,2022-05-09 13:30:55,-0.139696,0.301840,-0.015969,-0.016604,0.016212


In [35]:
import plotly.express as px

px.scatter(df_ndvi, x="date", y="NDVI_median")

In [34]:
px.scatter(df_ndci, x="date", y="NDCI_median")

As séries de NDVI e NDCI apresentam uma variação sazonal com maiores valores no verão e menores no inverno, condizente com o esperado. Destaca-se que a série de NDCI parece mais instável.

In [38]:
df_gee = df_ndvi.merge(df_ndci, on=["millis", "date"])

In [39]:
df_gee

,millis,date,NDVI_min,NDVI_max,NDVI_mean,NDVI_median,NDVI_stdDev,NDCI_min,NDCI_max,NDCI_mean,NDCI_median,NDCI_stdDev
0,1546090840000,2018-12-29 13:40:40,-0.784057,0.914034,-0.320348,-0.308634,0.129478,-0.167931,0.634409,-0.023233,-0.029342,0.034691
1,1548423047000,2019-01-25 13:30:47,-0.671004,0.923810,-0.394626,-0.402299,0.109197,-0.180560,0.633309,-0.005937,-0.005875,0.035793
2,1548682843000,2019-01-28 13:40:43,-0.578635,0.917371,-0.317525,-0.316366,0.086279,-0.158893,0.647395,-0.010855,-0.013683,0.033420
3,1549546843000,2019-02-07 13:40:43,-0.727873,0.924584,-0.425313,-0.417968,0.125853,-0.193239,0.638596,-0.030255,-0.029306,0.040513
4,1549719051000,2019-02-09 13:30:51,-0.995249,0.907998,-0.540698,-0.542969,0.130706,-0.176221,0.622495,-0.012386,-0.009748,0.045008
...,...,...,...,...,...,...,...,...,...,...,...,...
166,1650807045250,2022-04-24 13:30:45,-0.244416,0.699890,-0.170517,-0.173774,0.061692,-0.114764,0.291866,-0.018705,-0.020502,0.020489
167,1651239056300,2022-04-29 13:30:56,-0.260390,0.682045,-0.144146,-0.166080,0.077135,-0.111469,0.297750,-0.013373,-0.012695,0.021745
168,1651930843010,2022-05-07 13:40:43,-0.265924,0.681657,-0.141280,-0.154302,0.075562,-0.094701,0.294320,-0.009096,-0.008776,0.017226
169,1652103055120,2022-05-09 13:30:55,-0.327605,0.697630,-0.188140,-0.191566,0.059146,-0.139696,0.301840,-0.015969,-0.016604,0.016212


### 2.3. Cruzamento com dados locais

#### 2.3.1. RS Água

Estação 	87442000

In [43]:
rs_agua = pd.read_excel("../../data/external/rsagua/Dados_20220515181337.xls", sheet_name="Dados_Brutos")

In [47]:
def read_rs_agua(file_dir, ratio_censurado=0.5):
    """Faz a leitura de arquivo Excel oriundo do RS Água e transforma em formato long"""
    
    rs_agua_raw = pd.read_excel(file_dir, sheet_name="Dados_Brutos")
    
    rs_agua_raw.columns = rs_agua_raw.columns.\
        str.strip().str.lower().\
        str.replace(" ", "_").\
        str.replace("á", "a").\
        str.replace("ã", "a").\
        str.replace("ê", "e").\
        str.replace("é", "e").\
        str.replace("í", "i").\
        str.replace("ó", "o").\
        str.replace("ç", "c").\
        str.replace("__-_", "_").\
        str.replace("(", "").\
        str.replace(")", "")
    
    valores_fixos = rs_agua_raw[["indice",
                       "cod._estacao",
                       "latitude",
                       "longitude",
                       "bacia_hidrografica",
                       "recurso_hidrico",
                       "regiao",
                       "municipio",
                       "ambiente"]]
    
    resultados = rs_agua_raw.drop(valores_fixos, axis=1)
    
    resultados_long = pd.melt(resultados,
        id_vars=["data_coleta", "hora_coleta", "chuva_24h"],
        var_name="parametro",
        value_name="resultado")
    
    # flag de dados censurados
    resultados_long["censurado_esquerda"]=resultados_long["resultado"].str.contains("<")

    # remove valores especiais
    resultados_long["resultado"] = resultados_long.resultado.str.strip().str.replace(" ", "").\
        str.replace(",", ".").\
        str.replace(r"[A-Z]*", "").\
        str.replace("=", "").\
        str.replace("<", "")
    
    resultados_long["resultado"] = pd.to_numeric(resultados_long["resultado"])
    
    resultados_long.loc[
        resultados_long["censurado_esquerda"]==True,
                        "resultado"] = resultados_long.loc[resultados_long["censurado_esquerda"]==True,
                       "resultado"].apply(lambda x: x * ratio_censurado)
    
    return resultados_long

In [49]:
guaiba_fepam = read_rs_agua("../../data/external/rsagua/Dados_20220515181337.xls")

<ipython-input-47-ffaae50c2e21>:6: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

<ipython-input-47-ffaae50c2e21>:6: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

<ipython-input-47-ffaae50c2e21>:41: FutureWarning:

The default value of regex will change from True to False in a future version.



In [52]:
guaiba_fepam["parametro"].value_counts()

alcalinidade                             11
salinidade                               11
nitrogenio_amoniacal                     11
nitrogenio_orgânico                      11
nitrogenio_total_kjeldahl                11
oxigenio_dissolvido                      11
ph                                       11
profundidade_coleta                      11
profundidade_total                       11
solidos_dissolvidos_totais               11
niquel                                   11
solidos_suspensos_totais                 11
solidos_totais                           11
temperatura_da_agua                      11
temperatura_do_ar                        11
transparencia_da_agua                    11
turbidez                                 11
vazao_recurso_hidrico                    11
nitrato                                  11
mercúrio_em_micrograma_por_litro_ug/l    11
aluminio                                 11
condutividade                            11
cadmio                          

In [53]:
ciano_fepam = guaiba_fepam.loc[guaiba_fepam["parametro"]=="fitoplancton_cianobacterias"]
clorofila_fepam = guaiba_fepam.loc[guaiba_fepam["parametro"]=="clorofila_a"]

Cruzamento para dados da mesma semana

In [60]:
guaiba_fepam["week"] = pd.to_datetime(guaiba_fepam["data_coleta"]).dt.to_period("W")
ciano_fepam = guaiba_fepam.loc[guaiba_fepam["parametro"]=="fitoplancton_cianobacterias"]
clorofila_fepam = guaiba_fepam.loc[guaiba_fepam["parametro"]=="clorofila_a"]

df_gee["week"] = pd.to_datetime(df_gee["date"]).dt.to_period("W")
fepam_gee = pd.merge(clorofila_fepam, df_gee, on="week")

Apenas 3 cruzamentos

In [61]:
fepam_gee

,data_coleta,hora_coleta,chuva_24h,parametro,resultado,censurado_esquerda,week,millis,date,NDVI_min,NDVI_max,NDVI_mean,NDVI_median,NDVI_stdDev,NDCI_min,NDCI_max,NDCI_mean,NDCI_median,NDCI_stdDev
0,2019-02-13,2022-05-15 11:15:00,MÉDIA,clorofila_a,2.86,False,2019-02-11/2019-02-17,1550151047000,2019-02-14 13:30:47,-0.914530,0.927669,-0.579319,-0.589906,0.138667,-0.273986,0.653631,-0.046042,-0.044935,0.049084
1,2019-05-15,2022-05-15 10:06:00,FRACA,clorofila_a,0.04,True,2019-05-13/2019-05-19,1558186850000,2019-05-18 13:40:50,-0.969231,0.991597,-0.491666,-0.566438,0.209613,-0.428571,0.942857,-0.031008,-0.035188,0.048194
2,2019-11-13,2022-05-15 10:39:00,AUSENTE,clorofila_a,2.94,False,2019-11-11/2019-11-17,1573911049528,2019-11-16 13:30:49,-0.932773,0.917513,-0.470467,-0.503928,0.155848,-0.410409,0.644769,-0.030755,-0.034971,0.032933


#### 2.3.2. Vigiágua

In [78]:
import urllib

VIGILANCIA_URL = "https://sage.saude.gov.br/dados/sisagua/controle_mensal_demais_parametros.zip"
DOWNLOAD_DIR = os.path.join("..", "..", "data", "external", "vigilancia")
MUNICIPIO = "PORTO ALEGRE"
MANANCIAL = "GUAIBA"

def read_vigilancia(download_dir=DOWNLOAD_DIR, municipio=MUNICIPIO, manancial=MANANCIAL, url=VIGILANCIA_URL):
    """Realiza a leitura de dados baixados da Vigilancia (controle mensal demais parametros)"""
    
    filename = os.path.join(DOWNLOAD_DIR, "controle_mensal_demais_parametros.zip")
    
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename)
        
    vigilancia = pd.read_csv(filename,
           compression="zip",
            sep=";",
            decimal=",",
            encoding="latin-1",
            parse_dates=["Data de preenchimento do relatório mensal",
                        "Data da coleta"])
    
    vigilancia =  vigilancia.loc[( vigilancia["Município"] == municipio) &
          (vigilancia["Nome do manancial superficial"] == manancial), :]
    
    return vigilancia

In [79]:
vigi = read_vigilancia()

<ipython-input-78-4bb0d4ecd859>:16: DtypeWarning:

Columns (6,9,14,21,22,23,24,27) have mixed types. Specify dtype option on import or set low_memory=False.



In [85]:
vigi[["Parâmetro", "Unidade"]].value_counts()

Parâmetro         Unidade                   
Escherichia coli  E.coli/100mL                  1968
Cianobactérias    Total de cianobactérias       1337
                  Outro(s) gênero(s)*           1285
                  Planktothrix sp.                90
                  Cylindrospermopsis sp.          69
                  Pseudoanabaena sp.              63
Cianotoxinas      Microcistina (µg/L)             63
Cryptosporidium   oocistos/L                      38
Giardia           Cistos/L                        38
Cianobactérias    Microcystis sp.                 34
                  Aphanocapsa sp.                 24
Cianotoxinas      Saxitoxina (µg/L)               20
Cianobactérias    Dolichospermum sp.              14
Cianotoxinas      Cilindrospermopsina (µg/L)       8
Cianobactérias    Planktolyngbya sp.               5
                  Geitlerinema sp.                 2
                  Anabaena sp.                     2
                  Aphanothece sp.                  1
d

In [90]:
ciano_vigi = vigi.loc[(vigi["Parâmetro"]=="Cianobactérias") & (vigi["Unidade"]=="Total de cianobactérias")]
ciano_vigi["Nome da ETA / UTA"].value_counts()

MOINHOS DE VENTO          277
SÃO JOÃO                  275
JOSÉ LOUREIRO DA SILVA    264
BELÉM NOVO                261
TRISTEZA                  260
Name: Nome da ETA / UTA, dtype: int64

In [94]:
ciano_vigi["week"] = pd.to_datetime(ciano_vigi["Data da coleta"]).dt.to_period("W")

<ipython-input-94-dafa8d495715>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [105]:
vigi_gee = pd.merge(ciano_vigi.loc[:, ["Data da coleta","Nome da ETA / UTA", "Resultado", "week"]], df_gee, on="week")
vigi_gee = vigi_gee.loc[vigi_gee["Nome da ETA / UTA"]=="BELÉM NOVO"].reset_index(drop=True)

In [111]:
px.scatter(vigi_gee, x="NDVI_median", y="Resultado")

## 3. Extraindo série temporal somente para ponto de amostragem

### 3.1. Extraindo dados GEE

In [173]:
p_lon = -51.215679
p_lat = -30.012175
amostragem_ponto = ee.Geometry.Point(p_lon, p_lat)
amostragem_buffer = amostragem_ponto.buffer(75)

region_amostragem = ee.Geometry.Polygon(
[[[-51.21646818059103,-30.013957938688517],
 [-51.21572789090292,-30.013855747123724],
 [-51.21486958401815,-30.013298336735033],
 [-51.214097107821864,-30.011923377716094],
 [-51.21371086972372,-30.011217310272695],
 [-51.214086378985805,-30.01047407595533],
 [-51.215556229525966,-30.009888775010992],
 [-51.21617850201742,-30.01054839963767],
 [-51.21665057080404,-30.010920017214005],
 [-51.21725138562338,-30.012257828961395],
 [-51.2176161660494,-30.01333549752507],
 [-51.21691879170553,-30.013744265296616],
 [-51.21646818059103,-30.013957938688517]]])

In [174]:
def clp_region_amostragem(img):
    return img.clip(region_amostragem)

In [175]:
s2a_amostragem = ee.ImageCollection("COPERNICUS/S2_SR")\
    .filterBounds(amostragem_buffer)\
    .filter(ee.Filter.lte("CLOUDY_PIXEL_PERCENTAGE", 30))\
    .map(mask_s2a_clouds)\
    .map(clp_region_amostragem)\
    .map(add_s2a_ndci_ndvi).map(add_id_date)

In [176]:
df_ndvi = create_df(s2a_amostragem, "NDVI")

In [177]:
px.scatter(df_ndvi, x="date", y="NDVI_median")

In [178]:
df_ndci = create_df(s2a_amostragem, "NDCI")

In [179]:
px.scatter(df_ndci, x="date", y="NDCI_median")

In [180]:
df_gee = df_ndvi.merge(df_ndci, on=["millis", "date"])

In [181]:
df_gee

,millis,date,NDVI_min,NDVI_max,NDVI_mean,NDVI_median,NDVI_stdDev,NDCI_min,NDCI_max,NDCI_mean,NDCI_median,NDCI_stdDev
0,1545658842000,2018-12-24 13:40:42,-0.586489,0.031217,-0.277986,-0.279614,0.085956,-0.153374,0.104317,-0.041278,-0.043944,0.025978
1,1546090840000,2018-12-29 13:40:40,-0.253764,-0.186165,-0.225873,-0.228663,0.014526,-0.037541,-0.005307,-0.022225,-0.022047,0.005300
2,1547386845000,2019-01-13 13:40:45,-0.376233,0.100970,0.015251,0.020257,0.049139,-0.021277,0.077296,0.028922,0.029205,0.014447
3,1548423047000,2019-01-25 13:30:47,-0.591954,0.021592,-0.512259,-0.521400,0.085056,-0.143822,0.101890,-0.030640,-0.031646,0.020816
4,1548682843000,2019-01-28 13:40:43,-0.565017,0.015712,-0.438463,-0.450631,0.089356,-0.097502,0.170153,-0.034595,-0.036123,0.026164
...,...,...,...,...,...,...,...,...,...,...,...,...
193,1650807045250,2022-04-24 13:30:45,-0.137812,-0.094512,-0.128355,-0.129987,0.006810,-0.043509,-0.002809,-0.020015,-0.020393,0.003762
194,1651239056300,2022-04-29 13:30:56,-0.156018,-0.114633,-0.142452,-0.143674,0.006458,-0.043827,-0.017696,-0.028983,-0.029126,0.003615
195,1651930843010,2022-05-07 13:40:43,-0.193469,-0.010076,-0.113817,-0.110856,0.038607,-0.023400,0.029009,-0.007149,-0.007150,0.007053
196,1652103055120,2022-05-09 13:30:55,-0.185109,-0.064431,-0.170978,-0.173068,0.013165,-0.031863,0.018947,-0.017292,-0.017698,0.005595


### 3.2. Cruzando com dados da Viligância

In [182]:
captacao = ciano_vigi.loc[ciano_vigi["Nome da ETA / UTA"] == "MOINHOS DE VENTO",
                          ["Data da coleta","Nome da ETA / UTA", "Resultado", "week"]].reset_index(drop=True)

df_gee["week"] = pd.to_datetime(df_gee["date"]).dt.to_period("W")

captacao_gee = pd.merge(captacao, df_gee, on="week")

In [183]:
captacao

,Data da coleta,Nome da ETA / UTA,Resultado,week
0,2014-01-27 00:00:00,MOINHOS DE VENTO,42.0,2014-01-27/2014-02-02
1,2014-03-11 00:00:00,MOINHOS DE VENTO,79.0,2014-03-10/2014-03-16
2,2014-06-16 00:00:00,MOINHOS DE VENTO,380.0,2014-06-16/2014-06-22
3,2014-10-20 00:00:00,MOINHOS DE VENTO,133.0,2014-10-20/2014-10-26
4,2014-07-09 00:00:00,MOINHOS DE VENTO,14.0,2014-07-07/2014-07-13
...,...,...,...,...
272,2020-10-06 00:00:00,MOINHOS DE VENTO,216.0,2020-10-05/2020-10-11
273,2021-03-10 00:00:00,MOINHOS DE VENTO,10387.0,2021-03-08/2021-03-14
274,2022-03-03 00:00:00,MOINHOS DE VENTO,3077.0,2022-02-28/2022-03-06
275,2022-03-23 00:00:00,MOINHOS DE VENTO,3446.0,2022-03-21/2022-03-27


In [184]:
captacao.loc[pd.to_datetime(captacao["Data da coleta"]) > "2019-01-27"]

,Data da coleta,Nome da ETA / UTA,Resultado,week
25,2019-02-25 00:00:00,MOINHOS DE VENTO,298.0,2019-02-25/2019-03-03
26,2019-06-03 00:00:00,MOINHOS DE VENTO,102.0,2019-06-03/2019-06-09
27,2019-12-20 00:00:00,MOINHOS DE VENTO,0.0,2019-12-16/2019-12-22
28,2020-12-09 00:00:00,MOINHOS DE VENTO,1486.0,2020-12-07/2020-12-13
29,2021-01-26 00:00:00,MOINHOS DE VENTO,364.0,2021-01-25/2021-01-31
...,...,...,...,...
272,2020-10-06 00:00:00,MOINHOS DE VENTO,216.0,2020-10-05/2020-10-11
273,2021-03-10 00:00:00,MOINHOS DE VENTO,10387.0,2021-03-08/2021-03-14
274,2022-03-03 00:00:00,MOINHOS DE VENTO,3077.0,2022-02-28/2022-03-06
275,2022-03-23 00:00:00,MOINHOS DE VENTO,3446.0,2022-03-21/2022-03-27


In [185]:
px.scatter(captacao, x="Data da coleta", y="Resultado")

In [186]:
captacao_gee

,Data da coleta,Nome da ETA / UTA,Resultado,week,millis,date,NDVI_min,NDVI_max,NDVI_mean,NDVI_median,NDVI_stdDev,NDCI_min,NDCI_max,NDCI_mean,NDCI_median,NDCI_stdDev
0,2019-02-25 00:00:00,MOINHOS DE VENTO,298.0,2019-02-25/2019-03-03,1551447049000,2019-03-01 13:30:49,-0.828571,-0.573477,-0.763509,-0.774496,0.043055,-0.165597,-0.077844,-0.133148,-0.133983,0.014468
1,2019-12-20 00:00:00,MOINHOS DE VENTO,0.0,2019-12-16/2019-12-22,1576762841487,2019-12-19 13:40:41,-0.311755,-0.076738,-0.149319,-0.149737,0.034107,-0.074586,0.020568,-0.011791,-0.012080,0.013551
2,2020-12-09 00:00:00,MOINHOS DE VENTO,1486.0,2020-12-07/2020-12-13,1607607048511,2020-12-10 13:30:48,-0.524517,-0.394636,-0.447150,-0.441930,0.029317,-0.065366,0.046065,0.007673,0.006003,0.017724
3,2019-07-01 00:00:00,MOINHOS DE VENTO,600.0,2019-07-01/2019-07-07,1562506851691,2019-07-07 13:40:51,-0.709642,-0.541345,-0.673113,-0.677379,0.024047,-0.142647,-0.086957,-0.118189,-0.119036,0.009877
4,2019-11-18 00:00:00,MOINHOS DE VENTO,0.0,2019-11-18/2019-11-24,1574170844418,2019-11-19 13:40:44,-0.205027,-0.101992,-0.153193,-0.153176,0.013656,-0.035714,0.017448,-0.014060,-0.014227,0.007739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,2021-03-10 00:00:00,MOINHOS DE VENTO,10387.0,2021-03-08/2021-03-14,1615383050394,2021-03-10 13:30:50,-0.567164,-0.351779,-0.508083,-0.512071,0.037167,-0.070140,0.023576,-0.027926,-0.028439,0.013782
63,2021-03-10 00:00:00,MOINHOS DE VENTO,10387.0,2021-03-08/2021-03-14,1615642845931,2021-03-13 13:40:45,-0.544787,-0.020255,-0.444582,-0.435614,0.058292,-0.061074,0.173104,0.008578,0.010169,0.027803
64,2022-03-03 00:00:00,MOINHOS DE VENTO,3077.0,2022-02-28/2022-03-06,1646314850256,2022-03-03 13:40:50,-0.140201,-0.051784,-0.127099,-0.128173,0.009201,-0.031960,0.012504,-0.020226,-0.020644,0.005527
65,2022-03-03 00:00:00,MOINHOS DE VENTO,3077.0,2022-02-28/2022-03-06,1646487048426,2022-03-05 13:30:48,-0.129697,-0.041494,-0.117631,-0.119421,0.010159,-0.016254,0.058677,-0.000821,-0.001794,0.007323


In [187]:
px.scatter(captacao_gee, x="NDVI_median", y="Resultado")

In [188]:
import numpy as np

log_captacao = captacao_gee.copy()
log_captacao["log_ciano"] = np.log1p((log_captacao["Resultado"]).astype(float))
log_captacao["log_ndvi_median"] = np.log1p((log_captacao["NDVI_median"]).astype(float))

px.scatter(log_captacao, x="log_ndvi_median", y="log_ciano")

In [191]:
px.scatter(log_captacao.loc[log_captacao["log_ndvi_median"] > -4], x="log_ndvi_median", y="log_ciano")

In [193]:
captacao_gee.to_csv("../../data/processed/ndvi_ndci_vigi.csv", index=False)